In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers -qq
!pip install sentencepiece -qq
!pip install tokenizer -qq

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/util

##Load Dataset

In [ ]:

import numpy as np
import pandas as pd

train = pd.read_csv('/data/train.csv')
dev = pd.read_csv('/data/dev.csv')
test = pd.read_csv('/data/test.csv')


train['free_text'] = train['free_text'].astype(str)
X_train = train['free_text']
y_train = train["label_id"]

dev['free_text'] = dev['free_text'].astype(str)
X_valid = dev['free_text']
y_valid = dev['label_id']

test['free_text'] = test['free_text'].astype(str)
X_test = test['free_text']
y_test = test['label_id']


## Set Cuda

In [ ]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

torch.cuda.is_available()

False

# Extract feature by using PhoBERT

In [ ]:
import pandas as pd
from glob import glob

train_sentences = list(train['free_text'].values)
train_labels = list(train['label_id'].values)

dev_sentences = list(dev['free_text'].values)
dev_labels = list(dev['label_id'].values)

test_sentences = list(test['free_text'].values)
test_labels = list(test['label_id'].values)

Load tokenizer of PhoBERT

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
#choose max_length for phobert model based on the input length

max_length = 0
list_len=[]
for sentence in train_sentences:
    length = len(tokenizer.tokenize(sentence))
    list_len.append(length)

from collections import Counter
Counter(list_len).most_common(100)

[(4, 2271),
 (6, 2265),
 (5, 2232),
 (7, 2168),
 (3, 1943),
 (8, 1886),
 (9, 1731),
 (10, 1513),
 (2, 1380),
 (11, 1349),
 (12, 1183),
 (13, 1081),
 (1, 1073),
 (14, 931),
 (15, 829),
 (16, 659),
 (17, 612),
 (18, 559),
 (19, 489),
 (20, 435),
 (21, 371),
 (22, 349),
 (23, 344),
 (24, 300),
 (25, 299),
 (28, 221),
 (26, 209),
 (27, 203),
 (29, 186),
 (30, 182),
 (33, 169),
 (32, 154),
 (31, 150),
 (34, 135),
 (36, 132),
 (35, 113),
 (37, 110),
 (38, 110),
 (40, 108),
 (39, 103),
 (41, 95),
 (43, 80),
 (44, 79),
 (45, 78),
 (46, 71),
 (47, 70),
 (42, 68),
 (51, 66),
 (48, 63),
 (49, 60),
 (53, 56),
 (58, 55),
 (50, 51),
 (52, 51),
 (56, 50),
 (55, 48),
 (59, 48),
 (54, 46),
 (67, 43),
 (57, 42),
 (62, 38),
 (63, 36),
 (66, 33),
 (60, 33),
 (68, 33),
 (65, 33),
 (64, 31),
 (61, 31),
 (75, 28),
 (71, 25),
 (74, 25),
 (70, 24),
 (72, 22),
 (73, 20),
 (69, 20),
 (76, 20),
 (82, 19),
 (78, 18),
 (85, 16),
 (77, 15),
 (80, 15),
 (84, 13),
 (79, 13),
 (87, 13),
 (81, 12),
 (92, 11),
 (83, 9),


In [ ]:
# Encode train label

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_labels)
encoded_labels = le.transform(train_labels)
encoded_test_labels = le.transform(dev_labels)

In [ ]:
# Tokens IDs tensor

def encoder_generator(sentences,labels):

    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):

        sent_index.append(index)

        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=70,
                                             pad_to_max_length=True,
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0).cuda()
    attention_masks = torch.cat(attention_masks,dim=0).cuda()
    labels = torch.tensor(labels).cuda()
    sent_index = torch.tensor(sent_index).cuda()

    return sent_index,input_ids,attention_masks,labels

train_sent_index,train_input_ids,train_attention_masks,train_encoded_label_tensors = encoder_generator(train_sentences,encoded_labels)
dev_sent_index,dev_input_ids,dev_attention_masks,dev_encoded_label_tensors = encoder_generator(dev_sentences,encoded_test_labels)
print('Original: ', train_sentences[0])
print('Token IDs:', train_input_ids[0])

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# Connvert train, dev input by using TensorDataset

from torch.utils.data import TensorDataset,random_split

train_dataset = TensorDataset(train_input_ids,train_attention_masks,train_encoded_label_tensors)
dev_dataset = TensorDataset(dev_input_ids,dev_attention_masks,dev_encoded_label_tensors)

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(dev_dataset)))

In [ ]:
# Set cuda by using device

from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(dev_dataset,
                              sampler=RandomSampler(dev_dataset),
                              batch_size=bs)

Load model PhoBERT

In [ ]:
from transformers import AutoModel

phoBert = AutoModel.from_pretrained('vinai/phobert-base')
phoBert = phoBert.to(device)

# Build CNN

In [ ]:
# Build class CNN

import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):

        super().__init__()

        self.fc_input = nn.Linear(embedding_dim,embedding_dim)

        self.conv_0 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[0])

        self.conv_1 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[1])

        self.conv_2 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[2])

        self.conv_3 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[3])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, encoded):

        #embedded = [batch size, sent len, emb dim]
        embedded = self.fc_input(encoded)
        #print(embedded.shape)

        embedded = embedded.permute(0, 2, 1)
        #print(embedded.shape)

        #embedded = [batch size, emb dim, sent len]

        conved_0 = F.relu(self.conv_0(embedded))
        conved_1 = F.relu(self.conv_1(embedded))
        conved_2 = F.relu(self.conv_2(embedded))
        conved_3 = F.relu(self.conv_3(embedded))

        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        pooled_3 = F.max_pool1d(conved_3, conved_3.shape[2]).squeeze(2)

        #pooled_n = [batch size, n_fibatlters]

        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2, pooled_3), dim = 1).cuda())

        #cat = [batch size, n_filters * len(filter_sizes)]

        result =  self.fc(cat)

        #print(result.shape)

        return result

In [ ]:
# Hyperparameters

EMBEDDING_DIM = 768
N_FILTERS = 128
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = len(le.classes_)
DROPOUT = 0.3
PAD_IDX = tokenizer.pad_token_id

cnn = CNN(EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
cnn = cnn.to(device)

In [ ]:
# Optimizer and criterion

import torch.optim as optim

model_prameters = list(phoBert.parameters())+list(cnn.parameters())

optimizer = optim.Adam(model_prameters,lr=2e-5,eps=1e-8)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [ ]:
# Calculate accuracy per batch during train

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).cuda()

In [ ]:
# Def for training

from tqdm import tqdm

def train():

    epoch_loss = 0
    epoch_acc = 0

    phoBert.train()
    cnn.train()

    for batch in tqdm(train_data_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        optimizer.zero_grad()

        embedded = phoBert(b_input_ids,b_input_mask)[0]

        predictions = cnn(embedded)

        loss = criterion(predictions, b_labels)

        acc = categorical_accuracy(predictions, b_labels)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(train_data_loader), epoch_acc / len(train_data_loader)

In [ ]:
# Class for predict label

import numpy as np

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label

In [ ]:
# Evaluate loss, acc  and f1-macro

from sklearn.metrics import classification_report,accuracy_score,f1_score
def eval():
    epoch_loss = 0

    total_predictions = []
    total_true = []

    all_true_labels = []
    all_pred_labels = []

    phoBert.eval()
    cnn.eval()

    with torch.no_grad():

        for batch in tqdm(valid_data_loader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            embedded = phoBert(b_input_ids,b_input_mask)[0]
            predictions = cnn(embedded)

            loss = criterion(predictions, b_labels)
            epoch_loss += loss.item()

            predictions = predictions.detach().cpu().numpy()

            label_ids = b_labels.to('cpu').numpy()

            pred,true = predictions_labels(predictions,label_ids)

            all_pred_labels.extend(pred)
            all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')

    avg_val_loss = epoch_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))

    return avg_val_loss,avg_val_accuracy,macro_f1_score

In [ ]:
# Time for training

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# Set device and gpu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

phoBert.cuda()

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

# Training

In [ ]:
epochs = 20

for epoch in range(epochs):

    start_time = time.time()
    train_loss,train_acc = train()
    valid_loss,valid_acc,macro_f1 = eval()
    end_time = time.time()


    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    torch.save(phoBert,f'{epoch}_phobert_cnn_model_part1_'+'task2a_2.pt')
    torch.save(cnn,f'{epoch}_phobert_cnn_model_part2_'+'task2a_2.pt')
    print("model saved")

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. acc: {valid_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. F1: {macro_f1*100:.2f}%')
    print('=============Epoch Ended==============')

100%|██████████| 29/29 [00:13<00:00,  2.21it/s]


              precision    recall  f1-score   support

           0       0.97      0.91      0.94      3045
           1       0.16      0.85      0.27        62
           2       0.68      0.57      0.62       578

    accuracy                           0.86      3685
   macro avg       0.60      0.78      0.61      3685
weighted avg       0.91      0.86      0.88      3685

accuracy = 0.86
model saved
Epoch: 01 | Epoch Time: 6m 12s
	Train Loss: 0.518 | Train acc: 81.60%
	 Val. Loss: 0.374 |  Val. acc: 85.94%
	 Val. Loss: 0.374 |  Val. F1: 61.07%
=============Epoch Ended==============


100%|██████████| 29/29 [00:13<00:00,  2.22it/s]


              precision    recall  f1-score   support

           0       0.97      0.93      0.95      2993
           1       0.54      0.72      0.62       247
           2       0.66      0.71      0.68       445

    accuracy                           0.89      3685
   macro avg       0.72      0.79      0.75      3685
weighted avg       0.90      0.89      0.90      3685

accuracy = 0.89
model saved
Epoch: 02 | Epoch Time: 6m 16s
	Train Loss: 0.366 | Train acc: 86.86%
	 Val. Loss: 0.316 |  Val. acc: 89.01%
	 Val. Loss: 0.316 |  Val. F1: 75.02%
=============Epoch Ended==============


100%|██████████| 29/29 [00:13<00:00,  2.21it/s]


...

100%|██████████| 29/29 [00:13<00:00,  2.21it/s]


              precision    recall  f1-score   support

           0       0.93      0.95      0.94      2793
           1       0.62      0.62      0.62       330
           2       0.77      0.65      0.71       562

    accuracy                           0.88      3685
   macro avg       0.77      0.74      0.75      3685
weighted avg       0.87      0.88      0.88      3685

accuracy = 0.88
model saved
Epoch: 07 | Epoch Time: 6m 14s
	Train Loss: 0.158 | Train acc: 94.18%
	 Val. Loss: 0.386 |  Val. acc: 87.79%
	 Val. Loss: 0.386 |  Val. F1: 75.50%
=============Epoch Ended==============


100%|██████████| 29/29 [00:13<00:00,  2.21it/s]


              precision    recall  f1-score   support

           0       0.94      0.95      0.94      2848
           1       0.63      0.63      0.63       329
           2       0.72      0.68      0.70       508

    accuracy                           0.88      3685
   macro avg       0.76      0.75      0.76      3685
weighted avg       0.88      0.88      0.88      3685

accuracy = 0.88
model saved
Epoch: 08 | Epoch Time: 6m 14s
	Train Loss: 0.121 | Train acc: 95.70%
	 Val. Loss: 0.420 |  Val. acc: 88.09%
	 Val. Loss: 0.420 |  Val. F1: 75.57%
=============Epoch Ended==============


 36%|███▌      | 92/254 [02:10<03:49,  1.42s/it]

In [ ]:
epochs = 20

best_macro_f1 = float('0')

for epoch in range(epochs):

    start_time = time.time()
    train_loss,train_acc = train()
    valid_loss,valid_acc,macro_f1 = eval()
    end_time = time.time()


    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        torch.save(phoBert,'1_phobert_cnn_model_part1_'+'task2a_2.pt')
        torch.save(cnn,'1_phobert_cnn_model_part2_'+'task2a_2.pt')
        print("model saved")

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. acc: {valid_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. F1: {macro_f1*100:.2f}%')
    print('=============Epoch Ended==============')

100%|██████████| 29/29 [00:09<00:00,  3.12it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      0.92      0.93      2931
           1       0.00      0.00      0.00         0
           2       0.74      0.47      0.58       754

    accuracy                           0.83      3685
   macro avg       0.56      0.47      0.50      3685
weighted avg       0.90      0.83      0.86      3685

accuracy = 0.83
model saved
Epoch: 01 | Epoch Time: 4m 20s
	Train Loss: 0.552 | Train acc: 80.12%
	 Val. Loss: 0.436 |  Val. acc: 83.09%
	 Val. Loss: 0.436 |  Val. F1: 50.30%
=============Epoch Ended==============


100%|██████████| 29/29 [00:09<00:00,  3.11it/s]


              precision    recall  f1-score   support

           0       0.97      0.92      0.95      3012
           1       0.39      0.80      0.52       159
           2       0.68      0.63      0.66       514

    accuracy                           0.88      3685
   macro avg       0.68      0.79      0.71      3685
weighted avg       0.90      0.88      0.89      3685

accuracy = 0.88
model saved
Epoch: 02 | Epoch Time: 4m 27s
	Train Loss: 0.400 | Train acc: 85.69%
	 Val. Loss: 0.346 |  Val. acc: 87.84%
	 Val. Loss: 0.346 |  Val. F1: 70.74%
=============Epoch Ended==============


100%|██████████| 29/29 [00:09<00:00,  3.12it/s]


...

              precision    recall  f1-score   support

           0       0.93      0.94      0.94      2831
           1       0.63      0.55      0.59       379
           2       0.69      0.70      0.70       475

    accuracy                           0.87      3685
   macro avg       0.75      0.73      0.74      3685
weighted avg       0.87      0.87      0.87      3685

accuracy = 0.87
Epoch: 18 | Epoch Time: 4m 24s
	Train Loss: 0.038 | Train acc: 98.76%
	 Val. Loss: 0.567 |  Val. acc: 87.27%
	 Val. Loss: 0.567 |  Val. F1: 74.08%
=============Epoch Ended==============


100%|██████████| 29/29 [00:09<00:00,  3.12it/s]


              precision    recall  f1-score   support

           0       0.92      0.95      0.93      2788
           1       0.60      0.58      0.59       339
           2       0.74      0.63      0.68       558

    accuracy                           0.87      3685
   macro avg       0.75      0.72      0.73      3685
weighted avg       0.86      0.87      0.86      3685

accuracy = 0.87
Epoch: 19 | Epoch Time: 4m 24s
	Train Loss: 0.033 | Train acc: 98.97%
	 Val. Loss: 0.637 |  Val. acc: 86.68%
	 Val. Loss: 0.637 |  Val. F1: 73.42%
=============Epoch Ended==============


100%|██████████| 29/29 [00:09<00:00,  3.12it/s]

              precision    recall  f1-score   support

           0       0.91      0.95      0.93      2760
           1       0.64      0.57      0.60       366
           2       0.73      0.63      0.68       559

    accuracy                           0.87      3685
   macro avg       0.76      0.72      0.74      3685
weighted avg       0.86      0.87      0.86      3685

accuracy = 0.87
Epoch: 20 | Epoch Time: 4m 24s
	Train Loss: 0.030 | Train acc: 98.98%
	 Val. Loss: 0.647 |  Val. acc: 86.51%
	 Val. Loss: 0.647 |  Val. F1: 73.71%
=============Epoch Ended==============


In [ ]:
# Save PhoBERT and CNN

torch.save(phoBert,'module2_part1.pt')
torch.save(cnn,'module2_part2.pt')

## EVALUATING

In [ ]:
# Load phobert and cnn
# 1 -> 7
import torch
phoBert = torch.load(r'1_phobert_cnn_model_part1_task2a_2.pt', map_location='cpu')
cnn = torch.load(r'1_phobert_cnn_model_part2_task2a_2.pt', map_location='cpu')
phoBert.eval()
cnn.eval()


AttributeError: Can't get attribute 'CNN' on <module '__main__'>

Predict label from true label

In [ ]:
test_sent_index, test_input_ids, test_attention_masks, test_encoded_label_tensors = encoder_generator(test_sentences,test_labels)
test_dataset = TensorDataset(test_input_ids,test_attention_masks,test_encoded_label_tensors)

test_data_loader = DataLoader(test_dataset,
                              sampler=RandomSampler(test_dataset),
                              batch_size=bs)

all_pred_labels = []
all_true_labels = []

with torch.no_grad():
  for batch in tqdm(test_data_loader):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    embedded = phoBert(b_input_ids,b_input_mask)[0]
    predictions = cnn(embedded)


    predictions = predictions.detach().cpu().numpy()

    label_ids = b_labels.to('cpu').numpy()

    pred, true = predictions_labels(predictions, label_ids)

    all_pred_labels.extend(pred)
    all_true_labels.extend(true)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 73/73 [00:22<00:00,  3.31it/s]


In [ ]:
# The final score in the test set (classification report)

print(classification_report(all_pred_labels,all_true_labels, digits = 4))

              precision    recall  f1-score   support

           0     0.9719    0.9190    0.9447      7802
           1     0.5040    0.5511    0.5265       577
           2     0.5485    0.7610    0.6375       958

    accuracy                         0.8800      9337
   macro avg     0.6748    0.7437    0.7029      9337
weighted avg     0.8996    0.8800    0.8874      9337



In [ ]:
# Confusion matrix in thetest set

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(all_true_labels, all_pred_labels)
cm

array([[7170,   81,  126],
       [ 210,  318,  103],
       [ 422,  178,  729]])